# Olympic Dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# turn off pink boxes
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('athlete_events.csv')

In [3]:
df1 = pd.read_csv('noc_regions.csv')

In [4]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [5]:
df.shape

(271116, 15)

In [6]:
df.isnull().sum()

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

In [7]:
df1.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [8]:
df1.shape

(230, 3)

In [9]:
df1.isnull().sum()

NOC         0
region      3
notes     209
dtype: int64

In [10]:
def filter(x):
    if x == 'Gold' or x == 'Silver' or x =='Bronze':
        return 1
    else:
        return 0

In [11]:
df['medalist'] = df['Medal'].apply(filter)


In [12]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,medalist
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,0
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,0
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,0
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,1
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,0


In [34]:
df['BMI'] = round((df['Weight'] * 0.45359237) / ((df['Height'] / 100) ** 2), 1)

In [35]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,medalist,BMI
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,0,11.2
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,0,9.4
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,0,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,1,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,0,10.9


## Lets split our data

In [13]:
# train/validate/test split
train_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

## Lets look at the nulls in the first dataframe

In [14]:
df.shape

(271116, 16)

In [15]:
df.isnull().sum()

ID               0
Name             0
Sex              0
Age           9474
Height       60171
Weight       62875
Team             0
NOC              0
Games            0
Year             0
Season           0
City             0
Sport            0
Event            0
Medal       231333
medalist         0
dtype: int64

### lets look at age first

In [16]:
df.Age.describe()

count    261642.000000
mean         25.556898
std           6.393561
min          10.000000
25%          21.000000
50%          24.000000
75%          28.000000
max          97.000000
Name: Age, dtype: float64

## it appears there are likely some outliers, however it is likely safe to impute the nulls with the mean 

In [17]:
imputer = SimpleImputer(strategy='mean')
imputer = imputer.fit(train[['Age']])
train[['Age']] = imputer.transform(train[['Age']])

validate[['Age']] = imputer.transform(validate[['Age']])

test[['Age']] = imputer.transform(test[['Age']])

In [18]:
train.isnull().sum()

ID               0
Name             0
Sex              0
Age              0
Height       33482
Weight       35051
Team             0
NOC              0
Games            0
Year             0
Season           0
City             0
Sport            0
Event            0
Medal       129589
medalist         0
dtype: int64

In [19]:
train.Age.describe()

count    151824.000000
mean         25.554600
std           6.283664
min          11.000000
25%          22.000000
50%          25.000000
75%          28.000000
max          88.000000
Name: Age, dtype: float64

## Lets look at the next nulls 


In [20]:
train.isnull().sum()

ID               0
Name             0
Sex              0
Age              0
Height       33482
Weight       35051
Team             0
NOC              0
Games            0
Year             0
Season           0
City             0
Sport            0
Event            0
Medal       129589
medalist         0
dtype: int64

## I believe that height and weight nulls may be related to the year they played, so lets try that 


In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151824 entries, 177199 to 175071
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   ID        151824 non-null  int64  
 1   Name      151824 non-null  object 
 2   Sex       151824 non-null  object 
 3   Age       151824 non-null  float64
 4   Height    118342 non-null  float64
 5   Weight    116773 non-null  float64
 6   Team      151824 non-null  object 
 7   NOC       151824 non-null  object 
 8   Games     151824 non-null  object 
 9   Year      151824 non-null  int64  
 10  Season    151824 non-null  object 
 11  City      151824 non-null  object 
 12  Sport     151824 non-null  object 
 13  Event     151824 non-null  object 
 14  Medal     22235 non-null   object 
 15  medalist  151824 non-null  int64  
dtypes: float64(3), int64(3), object(10)
memory usage: 19.7+ MB


In [22]:
train.Year.value_counts()

1992    9140
1988    8245
2000    7906
2016    7702
1996    7672
2008    7612
2004    7536
2012    7218
1972    6808
1984    6419
1976    5942
1968    5825
1964    5259
1960    5248
1952    5212
1980    5038
1936    4133
1948    4107
1956    3596
1924    3145
1928    3124
2014    2741
2010    2463
2006    2426
1920    2395
2002    2320
1912    2253
1998    2038
1932    1882
1994    1747
1908    1743
1900    1011
1906     976
1904     725
1896     217
Name: Year, dtype: int64

In [23]:
train[(train['Year'] > 1960)].isnull().sum()

ID              0
Name            0
Sex             0
Age             0
Height       5696
Weight       6078
Team            0
NOC             0
Games           0
Year            0
Season          0
City            0
Sport           0
Event           0
Medal       96757
medalist        0
dtype: int64

## It appears some where between 1950 and 1960 they started recording the height and weight more frequently 

## Lets drop the games before the year 1960, as we don't have much data we really cant afford to have a null in the weight and height of the athlete performing 

In [24]:
train = train[(train['Year'] > 1960)]
validate = validate[(validate['Year'] > 1960)]
test = test[(test['Year'] > 1960)]

In [25]:
train.isnull().sum()

ID              0
Name            0
Sex             0
Age             0
Height       5696
Weight       6078
Team            0
NOC             0
Games           0
Year            0
Season          0
City            0
Sport           0
Event           0
Medal       96757
medalist        0
dtype: int64

In [26]:
train.shape

(112057, 16)